# DOAS Conversion
This script transforms the evaluation output of DOASIS into a useful format. We reduce to the useful columns and set sensible headers. Additionally we can set a useful timezone and transform the timestamp to a standard DateTime format.

In [1]:
require "narray"
require "daru"
include Daru
nil

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

# User Variables. 

In [2]:
prefix = ["20160205_NOx"]

dir = "../DOAS/"

# Used chemicals in Fit (for DOASIS use upper case)
# The concentrations + their error are retrieved and saved under
# lowercase chemical and lowecase chemical _err (e.g. NO2 becomes no2 and no2_err)
chemicals = ["NO2", "H2O", "O3", "CHOCHO"]

# Additional columns you want to get. Probably some sort of Index or timestamp
# In vectors give the name you would like to give them.
columns = ["Name", "StartDateAndTime"]
vectors = ["index", "timestamp"]

# Change Time Zone
change_tz = true
input_tz = "+01:00"
output_tz = 1.0/24

# Change Index. Newer Evaluation scripts changed the Inedex of the Spectrum from just a number to prefix+number
# This can be removed
change_index = true

# Probably good as is. 
raw_suffix = "_raw"
filetype = ".csv"

nil

In [3]:
def update_index(s)
  s.to_s.split("_").last.to_i  
end

:update_index

## Setup
Set the right filenames

In [4]:
if prefix.is_a?(String)
  prefix = [prefix]
end

unless prefix.is_a?(Array)
  raise "prefix needs to be a String or Array of Strings, but is a #{prefix.class}"
end
nil

Set the right column names according to DOASIS evaluation

In [5]:
chemicals.each do |chem|
  columns << "Fit Coefficient (" + chem + ")"
  columns << "Fit Coefficient Error (" + chem + ")"
  vectors << chem.downcase
  vectors << chem.downcase + "_err"
end
nil

## Conversion

In [6]:
start = DateTime.now

prefix.each do |p|
  
  input = dir + p + raw_suffix+ filetype
  output = dir + p + filetype
  
  d = DataFrame.from_csv(input)
  
  d = d[*columns]
  
  d.vectors = Index.new(vectors)
  
  if change_tz
    d["timestamp"] = d["timestamp"].collect do |t|
      DateTime.parse(t+input_tz).new_offset(output_tz)
    end
  end
  
  if change_index
    d["index"] = d["index"].collect do |s|
      update_index(s)
    end
  end
  
  d.write_csv(output)
  
  puts "Wrote #{output}"

end
  
stop = DateTime.now
delta = (stop - start).to_f*24*60

puts "Done. It took #{delta} minutes."

Wrote ../DOAS/20160205_NOx.csv
Done. It took 0.12729965816666666 minutes.
